In [1]:
import numpy as np
import pandas as pd
import urllib2,zipfile, shapefile
from json import dumps
import folium

# Helper functions

In [2]:
def download(url, target):
    response = urllib2.urlopen(url)
    with open(target,'wb') as output:
        output.write(response.read())

def shp2geojson(source, target):
    reader = shapefile.Reader(source)
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]
    buffer = []
    for sr in reader.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        buffer.append(dict(type="Feature", geometry=geom, properties=atr)) 
   
    geojson = open(target, "w")
    geojson.write(dumps({"type": "FeatureCollection","features": buffer}, indent=2, encoding='latin1') + "\n")
    geojson.close()

# Crèches et nombre de places

In [3]:
creches_file = 'http://data.caf.fr/dataset/6df7fc29-2031-4022-a557-3adeb19c744c/resource/729e9c45-15c3-4879-a1ce-1b45ae438f0a/download/ADRESSESEAJE2013.csv'
download(creches_file, 'creches.csv')
df_creches = pd.read_csv('creches.csv',sep=';')
df_creches.head()

,DFREFENR,STATUGPS,RAISOGPS,RAISOEEQ,NOMEQU,NUVOIEOU,TYVOIEOU,NOVOIEOU,COADREOU,LIDITEOU,NUCOMEOU,COPOSEOU,NOCOMEOU,NBPLA0A5
0,31/12/2013,Association loi 1901,IGESA INSTITUTION DE GESTION SOC DES ARMEES,IGESA ETS SOCIAUX MARINE,LES COCCINELLES - DONAUESCHING,NaN,R,Friedhofstrasse 22,NaN,NaN,XXXXX,XXXXX,DONAUESCHINGEN,23
1,31/12/2013,Association loi 1901,IGESA INSTITUTION DE GESTION SOC DES ARMEES,IGESA ETS SOCIAUX MARINE,LES GRENOUILLES - MULLHEIM,NaN,LD,Barenfelstrasse 14,NaN,NaN,XXXXX,XXXXX,MULLHEIM,12
2,31/12/2013,Association loi 1901,IGESA INSTITUTION DE GESTION SOC DES ARMEES,IGESA ETS SOCIAUX MARINE,LA SOURIS VERTE - VILLINGEN,NaN,R,kirnacherstrasse 34,NaN,NaN,XXXXX,XXXXX,VILLINGEN,10
3,31/12/2013,collectivit� locale,COMMUNE D'AMBERIEU EN BUGEY,ETS MULTI ACCUEIL ARC EN CIEL AMBERIEU,MULTI ACCUEIL ARC EN CIEL,NaN,R,du Clos Lebreton,NaN,NaN,01004,01500,AMBERIEU-EN-BUGEY,50
4,31/12/2013,Association loi 1901,ASS. CENTRE SOCIAL ET CULTUREL ARBENT,CENTRE SOCIAL ARBENT,HG ARBENT,1400,R,du G�n�ral ANDREA,BP 25,NaN,01014,01100,ARBENT,18


# Nombre de bébés

In [4]:
demo_file = 'http://www.insee.fr/fr/ppp/bases-de-donnees/donnees-detaillees/pop-sexe-age-quinquennal/pop-sexe-age-quinquennal6811.zip'
download(demo_file, 'demo_file.zip')
zipfile.ZipFile('demo_file.zip').extractall()

In [5]:
df_demo = pd.read_excel('pop-sexe-age-quinquennal6811.xls',sheetname='COM_2011', skiprows=13)
#df_naissance_arr = pd.read_excel('base_naissances_deces_2014.xls',sheetname='ARM', skiprows=5)
#df_naissance = pd.concat([df_naissance_communes, df_naissance_arr])
df_demo.head()

,RR,DR,CR,STABLE,DR13,LIBELLE,ageq_rec01s1rpop2011,ageq_rec01s2rpop2011,ageq_rec02s1rpop2011,ageq_rec02s2rpop2011,...,ageq_rec16s1rpop2011,ageq_rec16s2rpop2011,ageq_rec17s1rpop2011,ageq_rec17s2rpop2011,ageq_rec18s1rpop2011,ageq_rec18s2rpop2011,ageq_rec19s1rpop2011,ageq_rec19s2rpop2011,ageq_rec20s1rpop2011,ageq_rec20s2rpop2011
0,82,01,1,1,01,ABERGEMENT-CLEMENCIAT,20.000000,28.000000,36.000000,12.000000,...,4.000000,8.000000,8.000000,4.000000,4.000000,4.000000,0.000000,0.000000,0.00000,0.00000
1,82,01,2,1,01,ABERGEMENT-DE-VAREY,12.000000,4.000000,20.000000,4.000000,...,8.000000,8.000000,4.000000,8.000000,4.000000,4.000000,0.000000,0.000000,0.00000,0.00000
2,82,01,3,0,01,AMAREINS,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,82,01,4,1,01,AMBERIEU-EN-BUGEY,462.510750,588.593263,485.342761,449.061385,...,160.718969,302.385117,133.183525,250.345695,91.005063,165.689825,38.944495,44.631400,8.48699,11.23422
4,82,01,5,1,01,AMBERIEUX-EN-DOMBES,35.643564,63.366337,51.485149,23.762376,...,15.841584,23.762376,19.801980,15.841584,19.801980,3.960396,0.000000,7.920792,0.00000,0.00000


In [6]:
df_demo.DR = df_demo.DR.astype(str)
df_demo.CR = df_demo.CR.apply(lambda x:str(x).zfill(3))
df_demo['CODGEO'] = df_demo.DR + df_demo.CR
df_demo['NB_BB'] = df_demo.ageq_rec01s1rpop2011 + df_demo.ageq_rec01s2rpop2011

df_creches['DPT'] = df_creches.COPOSEOU.apply(lambda x:str(x)[0:2]) #TODO change this to get 2A, 974 etc ...

# Analyse départementale

In [7]:
df_demo_dpt = pd.DataFrame(df_demo.groupby('DR').NB_BB.sum())
df_demo_dpt.reset_index(inplace = True)

df_creches_dpt = pd.DataFrame(df_creches.groupby('DPT').NBPLA0A5.sum())
df_creches_dpt.reset_index(inplace = True)

df_dpt = pd.merge(df_demo_dpt, df_creches_dpt, left_on='DR', right_on='DPT', how='inner')
df_dpt['PLACE_PAR_BB'] = df_dpt.NBPLA0A5 / df_dpt.NB_BB
df_dpt.head()

,DR,NB_BB,DPT,NBPLA0A5,PLACE_PAR_BB
0,01,40980.494386,01,2620,0.063933
1,02,34557.910411,02,916,0.026506
2,03,16404.932452,03,838,0.051082
3,04,8062.622129,04,421,0.052216
4,05,7178.938219,05,525,0.073131


In [8]:
dpt_shp = 'http://osm13.openstreetmap.fr/~cquest/openfla/export/departements-20140306-100m-shp.zip'
download(dpt_shp, 'departements.zip')
zipfile.ZipFile('departements.zip').extractall()

In [9]:
shp2geojson('departements-20140306-100m.shp', 'departements.geojson')

In [11]:
#TODO: temove this crap
df_dpt2 = df_dpt.append(pd.DataFrame([['2A', 10,'2A',1,0.1]], columns=df_dpt.columns))
df_dpt2 = df_dpt2.append(pd.DataFrame([['2B', 10,'2B',1,0.1]], columns=df_dpt.columns))
df_dpt2 = df_dpt2.append(pd.DataFrame([['971', 10,'971',1,0.1]], columns=df_dpt.columns))
df_dpt2 = df_dpt2.append(pd.DataFrame([['972', 10,'972',1,0.1]], columns=df_dpt.columns))
df_dpt2 = df_dpt2.append(pd.DataFrame([['973', 10,'973',1,0.1]], columns=df_dpt.columns))
df_dpt2 = df_dpt2.append(pd.DataFrame([['974', 10,'974',1,0.1]], columns=df_dpt.columns))
df_dpt2 = df_dpt2.append(pd.DataFrame([['976', 10,'976',1,0.1]], columns=df_dpt.columns))

In [18]:
mapa = folium.Map(location=[45, 5],
                  tiles="Mapbox Bright",
                  zoom_start=5)
geo_path = r'departements.geojson'
mapa.choropleth(geo_path=geo_path, data=df_dpt2, columns=['DPT','PLACE_PAR_BB'], \
                key_on='feature.properties.code_insee', \
                fill_color='BuPu', fill_opacity=0.7, line_opacity=0.5)

mapa

/Users/eric/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:11: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.
